In [20]:
from openai import OpenAI
import json
import os

GPT_MODEL_3_5 = "gpt-3.5-turbo-1106"
GPT_MODEL_4 = "gpt-4-0125-preview"

OPEN_API_KEY = os.getenv("OPENAI_API_KEY")

client = OpenAI()
model = GPT_MODEL_3_5

def ask(prompt, client, model, temperature = 0):
    response = None
    
    response = client.chat.completions.create(
      model=model,
      messages=prompt,
      temperature=temperature,
      # response_format={ "type": "json_object" },
    )

    return (response.choices[0].message.content)

no_condition  = False
no_non_condition_flow = False
filter_flow = []

default_flow = "Basic flow"


In [21]:
def read_file_into_string(file_path):
    try:
        with open(file_path, 'r') as file:
            file_content = file.read()
        return file_content
    except FileNotFoundError:
        print(f"File '{file_path}' not found.")
        return None
    except Exception as e:
        print(f"Error reading file: {e}")
        return None


In [22]:
def write_to_file(filename, s):
    try:
        with open(filename, 'w') as file:
            file.write(s)
        print("Content has been written to", filename)
    except Exception as e:
        print("An error occurred:", e)

In [23]:
usecase_list = [
"""Search
Description
This feature allows users to enter search queries and view relevant product listings.
Actor: User
Preconditions: None
Postconditions: 
Basic Flow:
Step 1: User types keyword in the search bar.
Step 2: User press enter or Click search icon.
Step 3: System show list of relevant search match. Relevant search matchs are products with name or description including or related to the keywords. 
- Maximum 60 product cards are shown for each page.
- The pagination bar is at the bottom of the page, user can click on the desired page or left or right button to change to a new page.
Alterative flow: 
1. If the keyword related with a shop name; System display:
- the shop's information at the top of the search result including: shop name, number of items, ratings, and response rate, average response time
-shop product's card.
-related products card (product has name match search keyword).
2. If the keyword related with a brand name (without shop available); System display:
- the most related/best sold shop's that sell that brand at the top of the search result including: shop name, number of items, ratings, and response rate, average response time
-related products card (product has name match search keyword).
3. If the keyword not match any shop or brand; System display:
-related products card (product has name match search keyword).
4. User can filter the search result with 4 mode under the form of radio buttons: 
-result most related with the keyword (Default): Prior the exactly match by name, then partial match by name and then the match by description.
-newest product result.
-best seller result.
-sort the product by price (dropdown): from lowest to highest and from highest to lowest.
Exception flow:
If No Search Results Found, system display: random product cards.
""",
"""Purchase
Description
This feature allows users to purchase of items they have added to their shopping cart or at product detail page.
Actor: User
Preconditions: None
Postconditions: user can checkout order.
Basic Flow:
Step 1: User in the shopping cart page and User has added items to the shopping cart.
Step 2: User select items to checkout by selecting each items by clicking on checkbox before item.
Step 3: System displays the summary costs.
Step 4: User click the ""Checkout"" button.
Step 5: System process to checkout.
- User is redirected to page that shows one or many new orders of all the items user have chosen.
- For each product, there should be the information section of the product including name, options, quantity, price, which shop sells it, promotion information, the shipping information section, the platform's promotion information section and the payment section with the default payment method.
- In the order there should be default delivery location chosen by the system for user.
- Products from different shops will be separate into different orders, products from the same shop is in one order. 

Alterative flow: 
1. In shopping cart page: User can select all the items come from one store, user choose by clicking the checkbox at the head of the store.
2. User can purchase products in product detail page:
- User perform feature 'View a product detail'.
- User click button 'Buy Now'.
- If product has many options, user choose one available option before adding product to cart. 
- User can adjust quantity of product before adding product into cart by clicking on minus or plus button next to quantity of the product. 
- User click the ""Checkout"" button.
- System process to checkout.


Exception flow:
- user cannot click on the checkbox of the product that is out of stock or erased by the seller even though it is in user's shopping cart.
When user purchase products in product detail page:
- user cannot purchase product has many options without selecting one available option.
- user cannot purchase product with quantity that is bigger than the current stock or lower than one.
- User cannot purchase product with no stock or out of stock option of one product with multiple options.
- the button ""Checkout"" is disable if the selected product is invalid """,
"""Add item to cart
Description
This feature allows users to add items to their shopping cart for future purchase.
Actor: User
Preconditions: User is logged into their Shopee account.
Postconditions: 
The item is successfully added to the user's shopping cart.
The user can view and manage items in their shopping cart.
Basic Flow:
Step 1: User perform feature 'View a product detail'.
Step 2: User clicks the ""Add to Cart"" button.
Step 3: System confirms that the item has been added to the cart.
Step 5: System adds item to the cart and updates the cart icon to reflect the number of items in the cart.
Alterative flow: 
If product has many options, user choose one available option before adding product to cart.
User can adjust quantity of product before adding product into cart by clicking on minus or plus button next to quantity of the product. 

Exception flow: 
user cannot add product has many options without selecting one available option.
user cannot add product with quantity that is bigger than the current stock. If user tries to adjust the quantity over the maximum stock, the plus button is disabled and a message is shown bellow the quantity user tries to add ""You has reach maximum of the stock.""
If the quantity is at one, the minus button is disabled to prevent user adjust quantity below one. 
User cannot add to cart product with no stock, or out of stock option of one product with multiple options.
""",
"""Product Information Display
Precondition: User at homepage, search page, or cart page.
Postcondition: 
Basic Flow:
1. User click on a product card.
2. System show product's information included:
- Title and Description: 
        + Display the product title prominently.
        + Provide a detailed product description, including key features, usage instructions, and benefits.
- Images and Videos:
        + Showcase high-quality images of the product from multiple angles.
        + Include videos demonstrating the product's features and usage.
- Specifications:
        + List detailed product specifications (e.g., dimensions, weight, material, color options, etc.).
- Price Information
        + Display the current price of the product.
        + Highlight any discounts, offers, or special pricing available.
- Stock Availability
        + Show real-time stock availability for the product.
- Delivery Options:
        + Display available shipping methods and estimated delivery times.
        + Shipping Fees
- Seller Profile
        + Display the seller's profile, including their name, ratings, and reviews.
- Seller Contact
        + Provide an option to contact the seller directly through the platform's messaging system.
- Store Link
        + Link to the seller's store page on Shopee, allowing users to view other products from the same seller.
- Customer Reviews and Ratings:
        + Overall Rating: Display the average rating based on customer reviews.
        + Detailed Reviews: Show 3 latest detailed reviews in a list of detailed customer reviews, including ratings, written feedback, and user-uploaded images.

-Related Products
        + Display a list of related product cards.


Alterative flow: 
1. User can expand the 'Customer Reviews and Ratings' to view more reviews.
2. User can expand the 'Customer Questions' to view more questions and seller answers.
3. User can scroll to view more related product card,
4. User click on seller contact to contact the seller directly through the platform's messaging system.
5. User click on Store Link to view other products from the same seller.

""",
"""Checkout
Use Case ID: UC-002
Description: This feature allows users to complete the purchase of items in their shopping cart.
Actor: User

Preconditions: 
User is logged into their Shopee account.
Postconditions
The order is successfully placed.
The user receives an order confirmation.
Basic Flow
Step 1: User perform feature 'Purchase'.
Step 2: System displays the order summary, including items, quantities, prices, and total cost.
Step 3: System set shipping address as default address. User can change or add new a shipping address.
Step 4: System set default payment method as COD. There are other payment method such as: Shopee Pay, Google Pay, credit. User can change a payment method by select new payment method.
Step 5: User can add application voucher or shop voucher to order by select available voucher or enter voucher code.
Step 6: System set default shipping option as Standard Express. User can change shipping option.
Step 7: User can add note for shop.
Step 8: System update the total cost based on user changes. User reviews the order details.
Step 9: User clicks on the ""Place Order"" button.
Step 10: System processes the payment and confirms the order.
Step 11: System displays the order confirmation and details.
Alternative Flows:
1.If user has no default shipping address, system will force user to add a new shipping address.
2. If user want to change shipping address: 
-user click button change shipping address. system will display a list of saved address and user can select one.
3. If user want to add a new shipping address: 
-user click button change shipping address, add new address
-user fill the form: 
        + Full name: contains only letter and cannot be less than 2 letters
        + phone number: must be in format (+84) xxx xxx xxx with all the x is number
        + city: dropdown menu list cities provided by the platform
        + district: dropdown menu list districts provided by the platform
        + ward: dropdown menu list wards provided by the platform
        + location's address: 
        + Address type (optional, ratio): own house, working place
- user click button 'Finish' to add shipping address. 
4. if user enter a voucher code (for application voucher/ shop voucher):  
- user input a correct voucher code.
- user click 'apply' button.
- system add voucher to order and discount the order based on the voucher input.
5. If user change shipping option: 
- user click on shipping option change button.
- system display a list of available shipping options.
- user select one.

Exception Flow
1. user cannot add disable voucher.
2. user cannot apply an incorrect voucher code.
3. If the input is not match the require condition or the inputted location's address cannot be found in the map, system send notification and prevent user from adding new shipping address.
4. If there is an issue with the order:
System displays an error message and suggests corrective actions (e.g., updating the shipping address, checking payment details)."""
]


In [24]:
#Prompt
MAIN_FLOW_SYSTEM_PROMPT="""
I want you to act as software tester.
Your task is to read this information about one main flow of a use case.
Then you predict all scenarios that can happen in this flow.

Rules to predict scenarios:
- Stay close to the details described in the flow.
- Focus on important and likely scenarios, important scenario is the scenario that users are more likely to encounter it. 
- Minimize the appearance of rare scenarios. 
- If there is no other action in the flow beside clicking or there is no condition to vary the user's actions, that flow has one scenario only.
- A scenario often refers to a specific sequence of events or user actions that could potentially lead to a change in how the application behaves or responds.
- Test scenarios should be derived from cohesive sequences of steps that represent meaningful user interactions, rather than isolated steps.
- A scenario should cover from the first step to the final step in the flow, the start or the result of the scenario could be different.
- You cannot separate parts of a flow to be a scenario (Example: predict multiple scenarios for a flow by dividing steps into parts) because each scenarios should be independent and require a complete flow to proceed.
I only need scenarios's name for the output, I do not need the steps to go with it.
"""

SUB_FLOW_SYSTEM_PROMPT="""
I want you to act as software tester.
Your task is to read this information about one main flow and one alternative or exception flow of a use case.
Then you predict all scenarios that can lead user from the main flow to change to the alternative or exception flow mentioned for creating test cases.

Rules to predict scenarios:
- If there is no other action in the flow beside clicking or there is no condition to vary the user's actions, that flow has one scenario only.
- A scenario often refers to a specific sequence of events or user actions that could potentially lead to a change in how the application behaves or responds.
- Test scenarios should be derived from cohesive sequences of steps that represent meaningful user interactions, rather than isolated steps.
- A scenario should cover from the first step to the final step in the flow, the start or the result of the scenario could be different.
- You cannot separate parts of a flow to be a scenario (Example: predict multiple scenarios for a flow by dividing steps into parts) because each scenarios should be independent and require a complete flow to proceed.
- Do not generate scenarios with user analysis. (Example: User accidentally do A and user intentionally do A is the same scenario, so do not consider about "accidentally" or "intentionally" in scenario)
- Do not choose another option that is not chosen by the flow, eventhough it is mentioned (Example: A pop up with OK and Cancel, the flow only has step choose OK. Do not generate scenario that press Cancel)
- Do not generate scenario to test only the main flow.
I only need scenarios's name for the output, I do not need the steps to go with it.
"""

EXTRACT_CONDITION_SYSTEM_PROMPT="""
Given use case flows of a feature.
Your task is to identify all the interactive elements within the feature. 
For each interactive element:
Identify what type of that element (button,buttons, icon,scroll, text field,text area, tab, radio buttons, menu, combobox, sliders, switches, dialog, link, form,rating, filter).
Identify all the conditions mentioned in the use case of that element that would make the element valid and the conditions that would make the element invalid based on the description of the use case flow.
Do not arbitrarily create additional conditions that not mention in the use case flow.
Return the element extracted in json format.
The JSON format should follow the following structure:
{"Name of interactive element": {"condition": {valid:"conditions that make element valid", invalid: "conditions that make element invalid"}, "type": "element type"}}
Examples of output json format template: 
{"Username": {"condition": {"valid": "must be over 8 characters and below 30 characters, must be entered", racter, empty"}, "type": "text field"}}
{"Search button": {"condition": {"valid": "", "invalid": ""}, "type": "text field"}}
"""

GEN_SCENARIO_FOR_CONDITION_PROMPT="""
Given a list of interaction element for input value, their extracted conditions and corresponding use case.
For each given invalid condition:
- Generate a test scenario that test that condition.
Do not generate test scenario to test element/condition that not mention in the given element list.
Do not generate test scenario to test valid conditions.
I only need scenarios's name for the output, I do not need the steps to go with it.
"""
FILTER_SYSTEM_PROMPT="""
You will be provide with a use case and a list of test scenario.
Based on information in the use case flow, define what test scenario is necessary to test the use case and remove duplicate test scenarios.
Remove test case test system load error that not be mentioned in use case such as: Load Failure, System Error,Network Error,Non-Existent ...(and use case not mentioned these flow)
Make sure test scenarios filted cover all the flow of use case and every scenarios in the response is unique.
Return scenario name only.
"""


In [25]:
# Example usage:
# file_path = "D:\GPT-testing\SpecificationData\Adventura - Flow only\Adventura - Filter.txt"
for usecase in usecase_list:
    print(usecase)
    promptExtractCondtition = [
        { "role": "system", "content": SUB_FLOW_SYSTEM_PROMPT},
        { "role": "user", "content": usecase}
    ]

    gpt_response = ask(promptExtractCondtition, client, model)
    print(usecase)
    print("\all flow gen sub flow:"+gpt_response +"\n")


Search
Description
This feature allows users to enter search queries and view relevant product listings.
Actor: User
Preconditions: None
Postconditions: 
Basic Flow:
Step 1: User types keyword in the search bar.
Step 2: User press enter or Click search icon.
Step 3: System show list of relevant search match. Relevant search matchs are products with name or description including or related to the keywords. 
- Maximum 60 product cards are shown for each page.
- The pagination bar is at the bottom of the page, user can click on the desired page or left or right button to change to a new page.
Alterative flow: 
1. If the keyword related with a shop name; System display:
- the shop's information at the top of the search result including: shop name, number of items, ratings, and response rate, average response time
-shop product's card.
-related products card (product has name match search keyword).
2. If the keyword related with a brand name (without shop available); System display:
- the m

In [26]:
# Example usage:
# file_path = "D:\GPT-testing\SpecificationData\Adventura - Flow only\Adventura - Filter.txt"
for usecase in usecase_list:
    print(usecase)
    promptExtractCondtition = [
        { "role": "system", "content": SUB_FLOW_SYSTEM_PROMPT},
        { "role": "user", "content": usecase}
    ]

    gpt_response = ask(promptExtractCondtition, client, model)
    print("\all flow gen sub flow:"+gpt_response +"\n")


Search
Description
This feature allows users to enter search queries and view relevant product listings.
Actor: User
Preconditions: None
Postconditions: 
Basic Flow:
Step 1: User types keyword in the search bar.
Step 2: User press enter or Click search icon.
Step 3: System show list of relevant search match. Relevant search matchs are products with name or description including or related to the keywords. 
- Maximum 60 product cards are shown for each page.
- The pagination bar is at the bottom of the page, user can click on the desired page or left or right button to change to a new page.
Alterative flow: 
1. If the keyword related with a shop name; System display:
- the shop's information at the top of the search result including: shop name, number of items, ratings, and response rate, average response time
-shop product's card.
-related products card (product has name match search keyword).
2. If the keyword related with a brand name (without shop available); System display:
- the m

In [27]:
# Example usage:
# file_path = "D:\GPT-testing\SpecificationData\Adventura - Flow only\Adventura - Filter.txt"
for usecase in usecase_list:
    print(usecase)
    promptExtractCondtition = [
        { "role": "system", "content": SUB_FLOW_SYSTEM_PROMPT},
        { "role": "user", "content": usecase}
    ]

    gpt_response = ask(promptExtractCondtition, client, model)
    print(usecase)
    print("\all flow gen sub flow:"+gpt_response +"\n")


Search
Description
This feature allows users to enter search queries and view relevant product listings.
Actor: User
Preconditions: None
Postconditions: 
Basic Flow:
Step 1: User types keyword in the search bar.
Step 2: User press enter or Click search icon.
Step 3: System show list of relevant search match. Relevant search matchs are products with name or description including or related to the keywords. 
- Maximum 60 product cards are shown for each page.
- The pagination bar is at the bottom of the page, user can click on the desired page or left or right button to change to a new page.
Alterative flow: 
1. If the keyword related with a shop name; System display:
- the shop's information at the top of the search result including: shop name, number of items, ratings, and response rate, average response time
-shop product's card.
-related products card (product has name match search keyword).
2. If the keyword related with a brand name (without shop available); System display:
- the m